# 策略说明

如果股价沿趋势线上升，那么当其回到支撑线或者RSI处于低位时，可以买入

In [15]:
await init_notebook()
def feature_trendline(code, name, bars, results, ft):
    alt = altitude(bars)
    mf = MaLineFeatures()
    
    vec = mf.feature(bars[:-1], lines=6)
    f = mf.to_dict(vec)
    
    close = bars["close"]
    roc = close[-2:]/close[-3:-1]-1
    if f["trendline_slope"] > 0 and np.all(roc > 0):
        results.append((name, code, alt, f["trendline_slope"], f["trendline"], f["trendline_gap"], bars["close"][-1]))

In [4]:
def feature_parallel(code, name, bars, results, ft):
    alt = altitude(bars)
    mf = MaLineFeatures()
    
    vec = mf.feature(bars, lines=4)
    f = mf.to_dict(vec)
    
    if f["parallel"] > 0:
        results.append((name, code, alt, f["parallel"], bars[-1]["close"]))

In [16]:
results = await scan(feature_trendline, 139, nstocks=-1, silent=True)
df = pd.DataFrame(results, columns=["naem", "code", "alt", "slope", "trendline","gap", "price"])

2021-12-10
progress: 500/4024, results: 46, elapsed: 70, ETA: 493


2021-12-12 20:45:18,254 E 44910 asyncio:default_exception_handler:1738 | Task was destroyed but it is pending!
task: <Task pending name='Task-8412' coro=<RedisConnection._read_data() running at /Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/aioredis/connection.py:186> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x14eddb8b0>()]> cb=[RedisConnection.__init__.<locals>.<lambda>() at /Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/aioredis/connection.py:168]>
2021-12-12 20:45:18,254 E 44910 asyncio:default_exception_handler:1738 | Task was destroyed but it is pending!
task: <Task pending name='Task-8413' coro=<RedisConnection._read_data() running at /Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/aioredis/connection.py:186> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x14ef8e0a0>()]> cb=[RedisConnection.__init__.<locals>.<lambda>() at /Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packag

progress: 1000/4024, results: 101, elapsed: 72, ETA: 217
progress: 1500/4024, results: 180, elapsed: 74, ETA: 124
progress: 2000/4024, results: 235, elapsed: 76, ETA: 76
progress: 2500/4024, results: 287, elapsed: 187, ETA: 114
progress: 3000/4024, results: 336, elapsed: 189, ETA: 64
progress: 3500/4024, results: 376, elapsed: 197, ETA: 29
progress: 4000/4024, results: 457, elapsed: 290, ETA: 1


In [18]:
df[(df.trendline==5)&(df.gap<0.05)].nlargest(10, "slope")

,naem,code,alt,slope,trendline,gap,price
51,东华科技,002140.XSHE,0.653488,0.004254,5,0.035519,13.170000
178,万讯自控,300112.XSHE,0.942260,0.003802,5,0.044448,13.380000
310,士兰微,600460.XSHG,0.701800,0.003684,5,0.047222,63.000000
196,温州宏丰,300283.XSHE,0.583884,0.003286,5,0.041192,7.500000
219,昊志机电,300503.XSHE,0.512201,0.003035,5,0.049768,15.650000
375,新亚强,603155.XSHG,0.435502,0.002677,5,0.013636,57.299999
155,祥鑫科技,002965.XSHE,0.370809,0.002469,5,0.023525,25.850000
9,汇源通信,000586.XSHE,0.496487,0.002383,5,0.037093,8.880000
184,东富龙,300171.XSHE,0.487771,0.002358,5,0.030642,48.470001
59,东晶电子,002199.XSHE,0.556634,0.002081,5,0.016433,8.230000


In [3]:
results = await scan(feature_parallel, 60, nstocks=-1, silent=True)
df = pd.DataFrame(results, columns=["naem", "code", "alt", "parallel", "price"])

NameError: name 'feature_parallel' is not defined

In [ ]:
df[(df.price<5)&(df.parallel<5)&(df.parallel>2)]

In [ ]:
results = await scan(feature_cross, 129, nstocks=-1, silent=True)
df = pd.DataFrame(results, columns=["name", "code", "alt", "slope", "line", "price"])

# 均线黄金坑

In [ ]:
def ma(code, name, bars, results, ft):
    alt = altitude(bars)
    mf = MaLineFeatures()
    
    vec = mf.feature(bars, lines=5)
    results.append((code, name, *vec))

results = await scan(ma, 80, tm="2021-10-13", nstocks=-1)


In [23]:
cols = [col[0] for col in MaLineFeatures().columns_60]
df = pd.DataFrame(results, columns=["code", "name", *cols])
show_cols = ["name","trendline", "trendline_slope", "a5", "b5", "vx5", "a10", "b10", "vx10", "parallel_234"]
df[(df.trendline!=-1) & (df.trendline_slope>0) &(df.a5>5e-3) & (df.b5<-1e-2)&(df.parallel_234<21)][show_cols]

,name,trendline,trendline_slope,a5,b5,vx5,a10,b10,vx10,parallel_234
137,莱茵体育,3,0.000886,0.005193,-0.021173,4.961299,0.001921,-0.004713,5.773318,14
416,隆平高科,4,0.003498,0.006100,-0.018280,5.501658,0.001757,-0.007617,4.832124,11
768,太极股份,4,0.002821,0.006092,-0.030090,4.530245,0.001722,-0.007292,4.882620,16
938,宝鼎科技,2,0.007036,0.006763,-0.036812,4.278560,0.000903,0.000137,7.076120,12
979,金禾实业,4,0.001853,0.005697,-0.033224,4.083790,0.001678,-0.019300,1.250893,20
1428,中青宝,2,0.044968,0.006697,-0.043455,3.755814,0.003146,-0.049954,-0.939701,15
1547,四方达,4,0.004628,0.012691,-0.040914,5.387998,0.002094,-0.003658,6.126408,0
2827,京投发展,3,0.029661,0.009917,-0.100472,1.934315,-0.001196,-0.019096,14.983397,19
3525,司太立,2,0.010245,0.005927,-0.035516,4.003802,0.000504,-0.007374,-0.318720,0
3563,东方电缆,2,0.007162,0.006275,-0.038678,3.918238,-0.000313,0.006848,-3.945277,18


# 周均线支撑

In [57]:
bars = jq.get_bars("000875.XSHE", 100, '1w', end_dt='2021-12-03',df=False, include_now=True)
print(bars["close"])

[3.07 3.15 3.1  3.2  3.51 3.73 3.8  3.38 3.4  4.11 3.82 4.4  3.99 3.84
 3.72 3.4  3.49 3.83 3.71 3.84 3.86 3.67 3.53 3.59 3.52 3.47 3.52 3.69
 3.82 4.27 4.08 4.28 4.35 4.26 4.49 4.42 4.26 4.14 4.07 4.12 3.85 3.96
 4.12 4.12 3.98 3.87 3.89 3.84 3.9  4.06 4.02 3.88 4.05 4.56 4.27 4.1
 4.28 4.13 4.06 3.76 3.74 4.21 4.29 4.5  4.82 4.86 4.8  4.72 4.94 4.65
 4.54 4.61 4.75 4.97 5.18 5.41 5.1  5.08 4.84 5.83 5.46 5.46 5.55 5.29
 5.58 6.58 6.18 6.22 6.41 7.4  7.38 7.43 5.9  6.11 7.56 6.86 6.67 7.04
 7.07 7.34]


In [57]:
#def week_20_features(code, name, bars, results, ft):
bars = await get_bars("000875.XSHE", 100, '1w', '2021-12-03')
print(np.round(bars["close"],2)[19:])
print(bars["frame"][19:])
ma = moving_average(bars["close"], 20)
print(np.round(ma,2))

# align
bars = bars[19:]

close = bars["close"]
low = bars["close"]
frame = bars["frame"]

cross_points = np.argwhere((low*0.98 < ma) & (close > ma)).flatten()
print(frame[cross_points])

[3.52 3.47 3.52 3.69 3.82 4.27 4.08 4.28 4.26 4.49 4.42 4.26 4.14 4.07
 4.12 3.85 4.12 4.12 3.98 3.89 3.84 3.9  4.06 4.02 3.88 4.05 4.56 4.1
 4.28 4.13 4.06 3.76 3.74 4.21 4.29 4.5  4.82 4.86 4.8  4.72 4.94 4.65
 4.54 4.75 4.97 5.18 5.41 5.1  5.08 4.84 5.83 5.46 5.46 5.55 5.29 6.58
 6.18 6.22 6.41 7.4  7.38 5.9  6.11 7.56 6.86 6.67 7.04 7.07 8.  ]
[datetime.date(2020, 6, 5) datetime.date(2020, 6, 12)
 datetime.date(2020, 6, 19) datetime.date(2020, 6, 24)
 datetime.date(2020, 7, 3) datetime.date(2020, 7, 10)
 datetime.date(2020, 7, 17) datetime.date(2020, 7, 24)
 datetime.date(2020, 8, 7) datetime.date(2020, 8, 14)
 datetime.date(2020, 8, 21) datetime.date(2020, 8, 28)
 datetime.date(2020, 9, 4) datetime.date(2020, 9, 11)
 datetime.date(2020, 9, 18) datetime.date(2020, 9, 25)
 datetime.date(2020, 10, 9) datetime.date(2020, 10, 16)
 datetime.date(2020, 10, 23) datetime.date(2020, 11, 6)
 datetime.date(2020, 11, 13) datetime.date(2020, 11, 20)
 datetime.date(2020, 11, 27) datetime.date(20

In [85]:
bars_om = await get_bars("000875.XSHE", 24, '1w', '2021-12-03')
print(len(bars_om), bars_om["close"])
print(bars_om["frame"])

19 [5.83      5.46      5.46      5.55      5.29      6.5799994 6.1800003
 6.22      6.41      7.4       7.38      5.9       6.11      7.5600004
 6.8599997 6.6699996 7.04      7.07      8.       ]
[datetime.date(2021, 6, 25) datetime.date(2021, 7, 2)
 datetime.date(2021, 7, 9) datetime.date(2021, 7, 16)
 datetime.date(2021, 7, 23) datetime.date(2021, 8, 6)
 datetime.date(2021, 8, 13) datetime.date(2021, 8, 20)
 datetime.date(2021, 8, 27) datetime.date(2021, 9, 3)
 datetime.date(2021, 9, 10) datetime.date(2021, 10, 15)
 datetime.date(2021, 10, 22) datetime.date(2021, 10, 29)
 datetime.date(2021, 11, 5) datetime.date(2021, 11, 12)
 datetime.date(2021, 11, 19) datetime.date(2021, 11, 26)
 datetime.date(2021, 12, 3)]


In [87]:
bars_jq = jq.get_bars("000875.XSHE", 24, '1w', end_dt='2021-12-03 15:00:00', include_now=True, df=False)
print(bars_jq["date"])
np.round(moving_average(bars_jq["close"],20),2)

[datetime.date(2021, 6, 4) datetime.date(2021, 6, 11)
 datetime.date(2021, 6, 18) datetime.date(2021, 6, 25)
 datetime.date(2021, 7, 2) datetime.date(2021, 7, 9)
 datetime.date(2021, 7, 16) datetime.date(2021, 7, 23)
 datetime.date(2021, 7, 30) datetime.date(2021, 8, 6)
 datetime.date(2021, 8, 13) datetime.date(2021, 8, 20)
 datetime.date(2021, 8, 27) datetime.date(2021, 9, 3)
 datetime.date(2021, 9, 10) datetime.date(2021, 9, 15)
 datetime.date(2021, 10, 15) datetime.date(2021, 10, 22)
 datetime.date(2021, 10, 29) datetime.date(2021, 11, 5)
 datetime.date(2021, 11, 12) datetime.date(2021, 11, 19)
 datetime.date(2021, 11, 26) datetime.date(2021, 12, 3)]


array([6.11, 6.19, 6.29, 6.4 , 6.51])

In [42]:
standard = np.array([6.19, 6.29, 6.40, 6.51])

In [44]:
standard/ma[-4:]

array([1.01926561, 1.01944895, 1.02081506, 1.01283547])

# 30分钟支撑线

使用120支撑线，当RSI低位时买入

In [122]:
def feature_120_rsi(code, name, bars, results, ft):
    if len(bars) < 130:
        return
    
    close = bars["close"]
    
    isup = close[-1]/close[-2] - 1 > 0
    
    if not isup:
        return
    
    rsi = relative_strength_index(close, 6)[-3:]
    prsi = np.array([rsi30.get_proba(code, r) for r in rsi])
    
    if (prsi[0] and any(prsi < 0.1)):
        ma = moving_average(close, 120)[-7:]
        
        (a,b,_), pmae = polyfit(ma/ma[0])

        if abs(a) < 3e-3 and b > 5e-4 and pmae < 3e-4:
            results.append((code, name, bars[-1]["frame"], b, min(rsi), min(prsi)))
    elif any(rsi < 10):
        ma = moving_average(close, 120)[-7:]
        
        (a,b,_), pmae = polyfit(ma/ma[0])
        if abs(a) < 3e-3 and b > 5e-4 and pmae < 3e-4:
            results.append((code, name, bars[-1]["frame"], b, min(rsi), None))

In [128]:
results = await scan(feature_120_rsi, 140, '30m', nstocks=-1, tm="2021-12-06 10:00")

2021-12-06 10:00:00+08:00
progress: 500/4016, results: 1, elapsed: 3, ETA: 21
progress: 1000/4016, results: 3, elapsed: 5, ETA: 15
progress: 1500/4016, results: 4, elapsed: 7, ETA: 11
progress: 2000/4016, results: 5, elapsed: 9, ETA: 9
progress: 2500/4016, results: 5, elapsed: 16, ETA: 9
progress: 3000/4016, results: 7, elapsed: 18, ETA: 6
progress: 3500/4016, results: 10, elapsed: 20, ETA: 2
progress: 4000/4016, results: 12, elapsed: 22, ETA: 0


In [129]:
df = pd.DataFrame(results, columns=["code", "name", "frame", "b", "rsi", "prsi"])
df

,code,name,frame,b,rsi,prsi
0,000811.XSHE,冰轮环境,2021-12-06 10:00:00+08:00,0.000779,21.44,0.08
1,002264.XSHE,新华都,2021-12-06 10:00:00+08:00,0.002236,23.54,0.09
2,002270.XSHE,华明装备,2021-12-06 10:00:00+08:00,0.000816,24.51,0.08
3,300061.XSHE,旗天科技,2021-12-06 10:00:00+08:00,0.002779,20.90,0.09
4,300435.XSHE,中泰股份,2021-12-06 10:00:00+08:00,0.003115,23.21,0.08
5,600215.XSHG,长春经开,2021-12-06 10:00:00+08:00,0.002367,19.12,0.05
6,600235.XSHG,民丰特纸,2021-12-06 10:00:00+08:00,0.001748,20.38,0.08
7,601208.XSHG,东材科技,2021-12-06 10:00:00+08:00,0.000925,29.21,0.09
8,603013.XSHG,亚普股份,2021-12-06 10:00:00+08:00,0.001028,25.08,0.08
9,603028.XSHG,赛福天,2021-12-06 10:00:00+08:00,0.001526,12.07,0.01
